project_01_sk

In [73]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import scipy.stats as st

In [75]:
# 1.Explain the data. Check It is supervised or Unsupervised. Demonstrate?
# unsupervised learning does not uses labeled input and output data while supervised can do it.---
 # Supervised learning is a machine learning approach that's defined by its use of labeled datasets. 
 #These datasets are designed to train or “supervise” algorithms into classifying data or predicting outcomes accurately. 
 #Using labeled inputs and outputs, the model can measure its accuracy and learn over time.---

In [76]:
data=pd.read_csv('insurance.csv')

data

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
...,...,...,...,...,...,...,...
1333,50,male,30.970,3,no,northwest,10600.54830
1334,18,female,31.920,0,no,northeast,2205.98080
1335,18,female,36.850,0,no,southeast,1629.83350
1336,21,female,25.800,0,no,southwest,2007.94500


In [77]:
# 2.Estimate the minimum sample size n to get the 99% accurate predictions. (precision = 0.02)
#  z=-2.57  for(0.01/2) since two tailed from stat table

z=-2.57
e=0.02
p=0.5
q=1-p

samplesize_n= (z**2/e**2)*p*q
print(samplesize_n)

4128.062499999999


In [78]:
# 3. Check the data is cleaned or not. If not then clean it (Null values, Row/Column Duplicates, Outliers, Change the string into numbers)

data.isnull().sum()

age         0
sex         0
bmi         0
children    0
smoker      0
region      0
charges     0
dtype: int64

In [79]:
data.duplicated().sum()

1

In [80]:
duplicates=data[data.duplicated(keep='last')]
duplicates

,age,sex,bmi,children,smoker,region,charges
195,19,male,30.59,0,no,northwest,1639.5631


In [81]:
data.drop_duplicates(inplace = True)

In [82]:
data.duplicated().sum()

0

In [84]:

def outlier(colum):
    m= data[colum].mean()
    m1= data[colum].min()
    m2=data[colum].max()
    l= len(data)
    q1= data[colum].quantile(0.25)
    q3=data[colum].quantile(0.75)
    IQR= q3-q1
    lower= q1-(IQR*1.5)
    upper= q3+(IQR*1.5)
    n= len(data.loc[np.where((data[colum] > upper) | (data[colum] < lower))])
    perc= (n/l)*100
    print(f'{colum}\n percentage= {perc}\n number={n}\n mean= {m}\n min={m1}\n max={m2}')
    
for i in data.select_dtypes(exclude='object').columns:
    outlier(i)

age
 percentage= 0.0
 number=0
 mean= 39.222139117427076
 min=18
 max=64
bmi
 percentage= 0.6731488406881078
 number=9
 mean= 30.663451757666405
 min=15.96
 max=53.13
children
 percentage= 0.0
 number=0
 mean= 1.0957367240089753
 min=0
 max=5
charges
 percentage= 10.396409872849663
 number=139
 mean= 13279.121486655948
 min=1121.8739
 max=63770.42801


In [85]:
data.describe()

,age,bmi,children,charges
count,1337.000000,1337.000000,1337.000000,1337.000000
mean,39.222139,30.663452,1.095737,13279.121487
std,14.044333,6.100468,1.205571,12110.359656
min,18.000000,15.960000,0.000000,1121.873900
25%,27.000000,26.290000,0.000000,4746.344000
50%,39.000000,30.400000,1.000000,9386.161300
75%,51.000000,34.700000,2.000000,16657.717450
max,64.000000,53.130000,5.000000,63770.428010


In [86]:
data['sex']=data['sex'].map({'male':0,'female':1}).astype(int)
data['smoker']=data['smoker'].map({'yes':1,'no':0}).astype(int)
data['region']=data['region'].map({'southwest':1,'southeast':2,'northwest':3,'northeast':4}).astype(int)
data

,age,sex,bmi,children,smoker,region,charges
0,19,1,27.900,0,1,1,16884.92400
1,18,0,33.770,1,0,2,1725.55230
2,28,0,33.000,3,0,2,4449.46200
3,33,0,22.705,0,0,3,21984.47061
4,32,0,28.880,0,0,3,3866.85520
...,...,...,...,...,...,...,...
1333,50,0,30.970,3,0,3,10600.54830
1334,18,1,31.920,0,0,4,2205.98080
1335,18,1,36.850,0,0,2,1629.83350
1336,21,1,25.800,0,0,1,2007.94500


In [87]:
# 4. Check that sex and smoking are statistically independent or not.
column_1=data["sex"]
column_2=data["smoker"]
correlation=column_1.corr(column_2)
print(correlation)
# They both are independent.

-0.07659585039335874


In [88]:
# 5. Check that all regressor variables (independent variable) are independent of each other or not.
# Independent variables are age,sex,bmi,smoker and children.All regressor variables (independent variable) are independent of each other.
column_a = data["age"]
column_b = data["smoker"]
correlation = column_a.corr(column_b)
print(correlation)

-0.025587117601993975


In [50]:
x=data['age']
y=data['smoker']

xmean=data.age.mean()
data['diffx'] = xmean - x
data['diff_x^2'] = data.diffx**2
SSxx = data['diff_x^2'].sum()

ymean =y.mean()
data['diffy'] = ymean - y
SSxy = (data.diffx * data.diffy).sum()

m = SSxy/SSxx
print(m)

b=ymean- m*xmean
print(b)

def predicted_y(value):
    predict =m+value+b
    return predict

-0.0007188576016893378
0.23296752685128622


In [89]:
total_some_squares = ((y-ymean)**2).sum()
total_some_squares

217.88938714499255

In [94]:
y_hatt=predicted_y(x)
y_hatt

0       16903.923281
1        1743.551581
2        4477.461281
3       22017.469891
4        3898.854481
            ...     
1333    10650.547581
1334     2223.980081
1335     1647.832781
1336     2028.944281
1337    29202.359581
Name: age, Length: 1338, dtype: float64

In [97]:
some_residual_squares = ((y-y_hatt)**2).sum()
some_residual_squares

433214572971.3558

In [98]:
R_square=1-total_some_squares/some_residual_squares
R_square

0.9999999994970405

In [99]:
data.corr()

,age,sex,bmi,children,smoker,region,charges
age,1.000000,0.019814,0.109344,0.041536,-0.025587,-0.001626,0.298308
sex,0.019814,1.000000,-0.046397,-0.017848,-0.076596,0.004936,-0.058044
bmi,0.109344,-0.046397,1.000000,0.012755,0.003746,-0.157574,0.198401
children,0.041536,-0.017848,0.012755,1.000000,0.007331,-0.016258,0.067389
smoker,-0.025587,-0.076596,0.003746,0.007331,1.000000,0.002358,0.787234
region,-0.001626,0.004936,-0.157574,-0.016258,0.002358,1.000000,0.006547
charges,0.298308,-0.058044,0.198401,0.067389,0.787234,0.006547,1.000000


In [100]:
# 6. Check the dependency between response and regressors.
 # There small dependency on charges with respect to the(AGE,BMI) huge dependency on smoker.
column_a = data["age"]
column_b = data["charges"]
correlation = column_a.corr(column_b)
print(correlation)

0.2983082125097863


In [101]:
column_a = data["sex"]
column_b = data["charges"]
correlation = column_a.corr(column_b)
print(correlation)

-0.058044495790312914


In [102]:
column_a = data["bmi"]
column_b = data["charges"]
correlation = column_a.corr(column_b)
print(correlation)

0.19840083122624935


In [103]:
column_a = data["smoker"]
column_b = data["charges"]
correlation = column_a.corr(column_b)
print(correlation)

0.7872343672800326


In [104]:
data.corr()

,age,sex,bmi,children,smoker,region,charges
age,1.000000,0.019814,0.109344,0.041536,-0.025587,-0.001626,0.298308
sex,0.019814,1.000000,-0.046397,-0.017848,-0.076596,0.004936,-0.058044
bmi,0.109344,-0.046397,1.000000,0.012755,0.003746,-0.157574,0.198401
children,0.041536,-0.017848,0.012755,1.000000,0.007331,-0.016258,0.067389
smoker,-0.025587,-0.076596,0.003746,0.007331,1.000000,0.002358,0.787234
region,-0.001626,0.004936,-0.157574,-0.016258,0.002358,1.000000,0.006547
charges,0.298308,-0.058044,0.198401,0.067389,0.787234,0.006547,1.000000


In [107]:
# 7. Predict the regression Line to predict the charges for insurance using independent variables.
a1=data["age"]
a2=data["sex"]
a3=data["bmi"]
a4=data["smoker"]
b=data["charges"]

a1=np.array(a1)
a2=np.array(a2)
a3=np.array(a3)
a4=np.array(a4)
b=np.array(b)

n=len(a1)

x_ones= np.ones((n,1))

a1_new=np.reshape(a1,(n,1))
a2_new=np.reshape(a2,(n,1))
a3_new=np.reshape(a3,(n,1))
a4_new=np.reshape(a4,(n,1))

x_new=np.append(x_ones,a1_new,axis=1)
x_new=np.append(x_new,a2_new,axis=1)
x_new=np.append(x_new,a3_new,axis=1)
x_new=np.append(x_new,a4_new,axis=1)

print("x : ", x_new)
print()

x_new_transpose = np.transpose(x_new)

print("x Transpose : ", x_new_transpose)

x_new_transpose_dot_x_new = x_new_transpose.dot(x_new)
tempa =np.linalg.inv(x_new_transpose_dot_x_new)
tempb= x_new_transpose.dot(b)

Beeta_values = tempa.dot(tempb)
print("beta values:",Beeta_values)
print()

beta_0=Beeta_values[0]
beta_1=Beeta_values[1]
beta_2=Beeta_values[2]
beta_3=Beeta_values[3]
beta_4=Beeta_values[4]

print(beta_0)
print(beta_1)
print(beta_2)
print(beta_3)
print(beta_4)


def predicted_values(beta_0,beta_1,beta_2,beta_3,beta_4,age,sex,bmi,smoker):
    predicted_value = beta_0 +(age * beta_1)+ (sex * beta_2) + (bmi * beta_3) + (smoker * beta_4)
    return predicted_value


x :  [[ 1.   19.    1.   27.9   1.  ]
 [ 1.   18.    0.   33.77  0.  ]
 [ 1.   28.    0.   33.    0.  ]
 ...
 [ 1.   18.    1.   36.85  0.  ]
 [ 1.   21.    1.   25.8   0.  ]
 [ 1.   61.    1.   29.07  1.  ]]

x Transpose :  [[ 1.    1.    1.   ...  1.    1.    1.  ]
 [19.   18.   28.   ... 18.   21.   61.  ]
 [ 1.    0.    0.   ...  1.    1.    1.  ]
 [27.9  33.77 33.   ... 36.85 25.8  29.07]
 [ 1.    0.    0.   ...  0.    0.    1.  ]]
beta values: [-11736.2886855     259.34292641    106.89267306    323.06893696
  23832.22216519]

-11736.288685495267
259.342926414516
106.89267305788599
323.06893696491716
23832.222165186424


In [108]:
age=29
sex=1
bmi=28
smoker=1

print(predicted_values(beta_0,beta_1,beta_2,beta_3,beta_4,age,sex,bmi,smoker))

28769.701253787687


In [109]:
# 8. Predict the accuracy of the regression Model.
a=data.iloc[:,0:5].values
b=data.iloc[:,6].values
print("A: ",a)
print("B: ",b)

A:  [[19.    1.   27.9   0.    1.  ]
 [18.    0.   33.77  1.    0.  ]
 [28.    0.   33.    3.    0.  ]
 ...
 [18.    1.   36.85  0.    0.  ]
 [21.    1.   25.8   0.    0.  ]
 [61.    1.   29.07  0.    1.  ]]
B:  [16884.924   1725.5523  4449.462  ...  1629.8335  2007.945  29141.3603]


In [111]:
from sklearn.model_selection import train_test_split
a_train,a_test,b_train,b_test=train_test_split(a,b,test_size=0.25,random_state=0)

from sklearn.linear_model import LinearRegression
lin_regressor= LinearRegression()
lin_regressor.fit(a_train,b_train)
b_predict=lin_regressor.predict(a_test)
score=lin_regressor.score(a_test,b_test)
print("Linear Regression Accuracy is",score*100)

Linear Regression Accuracy is 74.37441203005137


In [135]:
# 9. Predict insurance charge for Age = 29, Sex = F, bmi = 28, children = 1, Smoke = Yes, region = southeast.
a1=data["age"]
a2=data["sex"]
a3=data["bmi"]
a4=data["smoker"]
a5=data["region"]
b=data["charges"]

a1=np.array(a1)
a2=np.array(a2)
a3=np.array(a3)
a4=np.array(a4)
a5=np.array(a5)

b=np.array(b)

n=len(a1)

x_ones= np.ones((n,1))

a1_new=np.reshape(a1,(n,1))
a2_new=np.reshape(a2,(n,1))
a3_new=np.reshape(a3,(n,1))
a4_new=np.reshape(a4,(n,1))
a5_new=np.reshape(a5,(n,1))


x_new=np.append(x_ones,a1_new,axis=1)
x_new=np.append(x_new,a2_new,axis=1)
x_new=np.append(x_new,a3_new,axis=1)
x_new=np.append(x_new,a4_new,axis=1)
x_new=np.append(x_new,a5_new,axis=1)


print("x : ", x_new)
print()

x_new_transpose = np.transpose(x_new)

print("x Transpose : ", x_new_transpose)

x_new_transpose_dot_x_new = x_new_transpose.dot(x_new)
tempa =np.linalg.inv(x_new_transpose_dot_x_new)
tempb= x_new_transpose.dot(b)

Beeta_values = tempa.dot(tempb)
print("beta values:",Beeta_values)
print()

beta_0=Beeta_values[0]
beta_1=Beeta_values[1]
beta_2=Beeta_values[2]
beta_3=Beeta_values[3]
beta_4=Beeta_values[4]
beta_5=Beeta_values[5]


print(beta_0)
print(beta_1)
print(beta_2)
print(beta_3)
print(beta_4)
print(beta_5)



def predicted_values(beta_0,beta_1,beta_2,beta_3,beta_4,beta_5,age,sex,bmi,smoker,region):
    predicted_value = beta_0 +(age * beta_1)+ (sex * beta_2) + (bmi * beta_3) + (smoker * beta_4) + (region *beta_5)
    return predicted_value



x :  [[ 1.   19.    1.   27.9   1.    1.  ]
 [ 1.   18.    0.   33.77  0.    2.  ]
 [ 1.   28.    0.   33.    0.    2.  ]
 ...
 [ 1.   18.    1.   36.85  0.    2.  ]
 [ 1.   21.    1.   25.8   0.    1.  ]
 [ 1.   61.    1.   29.07  1.    3.  ]]

x Transpose :  [[ 1.    1.    1.   ...  1.    1.    1.  ]
 [19.   18.   28.   ... 18.   21.   61.  ]
 [ 1.    0.    0.   ...  1.    1.    1.  ]
 [27.9  33.77 33.   ... 36.85 25.8  29.07]
 [ 1.    0.    0.   ...  0.    0.    1.  ]
 [ 1.    2.    2.   ...  2.    1.    3.  ]]
beta values: [-12885.51500278    258.90892289    108.83060914    333.06366081
  23829.22083425    345.99823508]

-12885.515002776927
258.9089228865528
108.83060914489761
333.06366081469605
23829.220834248717
345.99823508417467


In [121]:
age=29
sex=1
bmi=28
smoker=1
region=2

print(predicted_values(beta_0,beta_1,beta_2,beta_3,beta_4,beta_5,age,sex,bmi,smoker,region))

28578.67417730656


In [124]:
# 10. Give the percentage of error in regression model.

a= data['charges'].sum()
a_bar = data.charges.mean()
summation = 0  
n =data.charges.size 

difference = a - a_bar
squared_difference = difference**2  
summation = summation + squared_difference 
MSE = summation/n  
    
print("The Mean Square Error is: " , MSE)

The Mean Square Error is:  235407446944.19275


In [125]:
# 11. Give the 95% confidence interval for average charge insurance.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import statsmodels.stats.weightstats as sms
from scipy.stats import t
import math

In [126]:
data

,age,sex,bmi,children,smoker,region,charges
0,19,1,27.900,0,1,1,16884.92400
1,18,0,33.770,1,0,2,1725.55230
2,28,0,33.000,3,0,2,4449.46200
3,33,0,22.705,0,0,3,21984.47061
4,32,0,28.880,0,0,3,3866.85520
...,...,...,...,...,...,...,...
1333,50,0,30.970,3,0,3,10600.54830
1334,18,1,31.920,0,0,4,2205.98080
1335,18,1,36.850,0,0,2,1629.83350
1336,21,1,25.800,0,0,1,2007.94500


In [127]:
n=1337
x_bar =data.charges.mean()
x_bar

13279.121486655948

In [128]:
m= data.charges.median()
m

9386.1613

In [129]:
Sd = data.charges.std()
Sd

12110.359656344179

In [131]:
n = 1337
m_0 = m
alpha = 0.05
t_stat = math.sqrt(n)*(x_bar - m_0)/Sd
crit = t.ppf(1-alpha, n-1)
wing_span = crit*Sd/(math.sqrt(n))
print(("tstatistics_t = {:.4f}").format(t_stat))
print(("Confidence interval with 99% confidence level: (-∞)({:.4f})").format(x_bar+wing_span))

tstatistics_t = 11.7541
Confidence interval with 99% confidence level: (-∞)(13824.2763)
